# 1. 엑셀데이터 불러오기

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re


df_input = pd.read_csv('phone_info.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

# 2. 리뷰크롤링

In [6]:
import time
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델영문명
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": "",                        #리뷰내용 
    "URL":""                          #리뷰 URL
}

crawl_data = pd.DataFrame(my_dict,index=[0])
crawl_data_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    else:
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000204&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_input.iloc[index][4]+url2)
    driver.implicitly_wait(3)
    
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴 => 5개 넘어가면 검색정확도 떨어져서 고정
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
            pass
        else:
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") 
            market = area1[3].text 
            write_id = area1[4].text 
            upload_day = area1[5].text 
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            my_dict = {
                "pl_id": df_input.iloc[index][0],              #기종아이디(자체부여한것)
                "pl_model_code": df_input.iloc[index][3],      #모델코드
                "pl_name": df_input.iloc[index][4],            #모델영문명

                "star": star,                       #별점
                "market": market,                   #구입처(ex 인터파크, 11번가)
                "write_id": write_id,               #작성자(일부가림처리)
                "upload_day": upload_day,           #업로드날짜
                "title":title,                      #리뷰제목 
                "text": text,                       #리뷰내용 
                "URL": driver.current_url
            }
            
            if my_dict is None or my_dict == None:
                pass
            else:
                a = pd.DataFrame(my_dict,index=[0])
                crawl_data = crawl_data.append(a)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    #display(crawl_data)
    return crawl_data

# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('../chromedriver')
driver.implicitly_wait(3)


for i in range(1):
#     print("=================",i,"skt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"skt")) 
#     print("=================",i,"kt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"kt"))
#     print("=================",i,"lg","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"lg"))
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"all"))
    
crawl_data = crawl_data[crawl_data['pl_id'] != '']
crawl_data = crawl_data.reset_index(drop=True)

    
    
    

In [8]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
display(crawl_data)

pandas version:  1.0.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text,URL
0,1,A2221,아이폰11,5,하이마트쇼핑몰,z*o*****,21.02.13.,토요일 배송 될거라 생각했는데,토요일 배송 될거라 생각했는데 월요일 배송되었네요.아이 졸업 선물로 카메라 좋은 거...,https://search.shopping.naver.com/catalog/2085...
1,1,A2221,아이폰11,4,스마트베이비,ciao****,21.01.10.,"실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...","실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...",https://search.shopping.naver.com/catalog/2085...
2,1,A2221,아이폰11,4,11번가,lo******,20.08.09.,좋아요,"배송빠르고 좋았어요.단, 아이폰이 스크래치있는 초기불량제품이어서 뽑기를 잘못한듯.매...",https://search.shopping.naver.com/catalog/2085...
3,1,A2221,아이폰11,5,G마켓,k*0*****,20.11.21.,완전 적극적으로 추천합니다,골드로 주문하였는데 품절이라 미드나잇으로 보내주셨습니다~ 추가금 없이 업그레이드 받...,https://search.shopping.naver.com/catalog/2085...
4,1,A2221,아이폰11,5,애플 공식 브랜드스토어,1004****,20.10.04.,"구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...","구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...",https://search.shopping.naver.com/catalog/2085...
5,1,A2221,아이폰11,5,프렌드가젯,flow****,20.11.29.,여러군데 알아보고 어렵게 샀는데요 궁금한 점도 빠른 답변으로 응대해주시고 상품도 정...,여러군데 알아보고 어렵게 샀는데요 궁금한 점도 빠른 답변으로 응대해주시고 상품도 정...,https://search.shopping.naver.com/catalog/2085...
6,1,A2221,아이폰11,3,하이마트쇼핑몰,k*n*****,20.07.08.,아.. 진짜 딥빡이네요\n스마트픽 하,아.. 진짜 딥빡이네요스마트픽 하지마세요.개인간 택배거래해도 물건 뽁뽁이에 싸주는게...,https://search.shopping.naver.com/catalog/2085...
7,1,A2221,아이폰11,5,SSG닷컴,lo******,20.02.07.,테두리: 지문 묻는 실버 뒷면:,테두리: 지문 묻는 실버 뒷면: 지문 묻지 않는 화이트 크기: 손에 착 감기는 맛용...,https://search.shopping.naver.com/catalog/2085...
8,1,A2221,아이폰11,4,위메프,pik*****,20.07.28.,아이폰 12살까하다가,아이폰 12살까하다가11pro 구매했습니다 ㅎㅎ 이쁩니다알뜰폰으로 같이 사용하려고 ...,https://search.shopping.naver.com/catalog/2085...
9,1,A2221,아이폰11,5,SSG닷컴,lj******,19.12.27.,카드 할인으로 15만원 할인 받고,카드 할인으로 15만원 할인 받고 제가 원하는 색상과 용량을 구매해서 너무 만족합니...,https://search.shopping.naver.com/catalog/2085...


In [9]:
crawl_data.to_csv('review_data1.csv.csv',encoding='utf-8-sig')

# 엑셀에서 pl_id 오름차순, text_len(추가후) 내림차순 정렬하고 난 후 => file.csv

In [138]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                    #모델영문명
    "text1": "",
    "text2": "",
    "text3": ""
}

filtered_reivew = pd.DataFrame(my_dict,index=[0])
filtered_reivew_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

df_output = pd.read_csv('file.csv')
pl_ids = df_output['pl_id'].unique()
models = []
for pl_id in pl_ids:
    is_name = df_output['pl_id'] == pl_id
    model = df_output[is_name]
    model = model[model.text.str.contains(
        r'(용량)|(색)|(사이즈)|(인치)|(화면)|(무게)|(센치)|(배터리)|(카메라)|(메모리)|(성능)|(출시)|(속도)'
        + r'|([0-9]+기가)|(보급형)|(방수)|(무선충전)|(지문인식)|(스크린)|(디스플레이)|(스펙)|(내장)'
        + r'|(외장)|(컬러)|(버튼)|(단자)|(현상)|(듀얼)|(화소)|(내구성)|(발열)|(업데이트)|(C타입)'
    )]    
    
    my_dict = {
    "pl_id": pl_id,                 
    }
    
    if len(model) > 0:
        my_dict["text1"] = model.iloc[0]['text']
    if len(model) > 1:
        my_dict["text2"] = model.iloc[1]['text']
    if len(model) > 2:
        my_dict["text3"] = model.iloc[2]['text']
    
    filtered_reivew = filtered_reivew.append(pd.DataFrame(my_dict,index=[0]))

In [139]:
filtered_reivew

,pl_id,text1,text2,text3
0,,,,
0,1,생각보다 하루 늦었지만 그래도 다른 곳보다 훨씬 빠르고 좋아요!!! 할인 받아서 저...,동생 사다줬는데 매우 좋아했습니다 특히 색이 너무 이쁘고 그때 마침 4만원인가 세일...,아이폰11 퍼플색을 살려고 통신사에 갔습니다. 하지만 퍼플이나 인기 좋은 제품의 색...
0,2,추석연휴 전 주라 배송이 좀 느리긴했네요 - 그래도 콜센터에서 전화와서 기분좋게 기...,한달동안 사용해 보면서 너무 너무 만족하면서 사용하고 있어요!!!! 아이폰은 흔히 ...,배송도 바로 다음날 받아볼 수 있고 아이폰11pro를 염두해두셨다면 여기서 구입하시...
0,3,백오십에 육박하는 상품을 택배로 받으려니 덜덜...상자에 심플하게 담겨옴. 에어캡이...,XS 쓰다가 넘어왔는데 너무 좋습니다 !특히 터치 후 빠른 반응 제일 만족하고 기존...,저렴하게 구매했습니다 공식인증 애플스토어에서 판매해서 구매하는데 믿음이 갔습니다 어...
0,4,친절하시구 배송도 빠르고 개통도 후개통인데 바로 됐고 가격도 방문하는거 제외 인터넷...,스그 오래걸릴줄알았는데 다행히 월요일에 구매해서 목요일에 받았어용 처음에 온할때 화...,너무 친절하시고 처리도 정말 빨라요..! 해피콜받고 바로 다음날 제품 받아볼 수 있...
...,...,...,...,...
0,245,싸게 잘 샀습니다. 다만 핸드폰 뒷면이 강화유리로 되어있어서 떨어뜨렸는데 벌써 깨어...,갤럭시 A7 2017년 제품 잘 받았습니다. 제품 색상이 없어서 아쉽긴 했지만 저렴...,NaN
0,246,NaN,NaN,NaN
0,247,NaN,NaN,NaN
0,248,NaN,NaN,NaN


In [141]:
filtered_reivew.to_csv('filtered_reivew.csv',encoding='utf-8-sig')

In [7]:
crawl_data

,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text,URL
0,1,A2221,아이폰11,5,하이마트쇼핑몰,z*o*****,21.02.13.,토요일 배송 될거라 생각했는데,토요일 배송 될거라 생각했는데 월요일 배송되었네요.아이 졸업 선물로 카메라 좋은 거...,https://search.shopping.naver.com/catalog/2085...
1,1,A2221,아이폰11,4,스마트베이비,ciao****,21.01.10.,"실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...","실버없어서 골드로 주문했는데, 기존 가지고있는거랑 비슷하지만 맘에 들어요~ 통신사 ...",https://search.shopping.naver.com/catalog/2085...
2,1,A2221,아이폰11,4,11번가,lo******,20.08.09.,좋아요,"배송빠르고 좋았어요.단, 아이폰이 스크래치있는 초기불량제품이어서 뽑기를 잘못한듯.매...",https://search.shopping.naver.com/catalog/2085...
3,1,A2221,아이폰11,5,G마켓,k*0*****,20.11.21.,완전 적극적으로 추천합니다,골드로 주문하였는데 품절이라 미드나잇으로 보내주셨습니다~ 추가금 없이 업그레이드 받...,https://search.shopping.naver.com/catalog/2085...
4,1,A2221,아이폰11,5,애플 공식 브랜드스토어,1004****,20.10.04.,"구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...","구매 후 2틀만에 도착했습니다. 출고가와 많이 차이나진 않지만, 일반 휴대폰 매장에...",https://search.shopping.naver.com/catalog/2085...
...,...,...,...,...,...,...,...,...,...,...
84,1,A2221,아이폰11,5,GS River,mono****,21.03.03.,완전 잘쓸것 같아요예쁜사진찍어야지,완전 잘쓸것 같아요예쁜사진찍어야지,https://search.shopping.naver.com/catalog/2085...
85,1,A2221,아이폰11,5,애플 공식 브랜드스토어,vip4****,20.11.14.,화이트 너무 예쁘고 깔끔해요배송도 빨라서 좋아요 월요일 받을줄알았는데 토요일에 받아...,화이트 너무 예쁘고 깔끔해요배송도 빨라서 좋아요 월요일 받을줄알았는데 토요일에 받아...,https://search.shopping.naver.com/catalog/2085...
86,1,A2221,아이폰11,5,애플 공식 브랜드스토어,kjk9****,20.10.28.,잘 받았습니다 배송도 빠르고 케이스에 필림도 보내주시고 번창 하십쇼,잘 받았습니다 배송도 빠르고 케이스에 필림도 보내주시고 번창 하십쇼,https://search.shopping.naver.com/catalog/2085...
87,1,A2221,아이폰11,5,애플 공식 브랜드스토어,0104****,20.10.31.,자급제 처음인데 진작에 이렇게 살껄 그랬어요 너무 좋아요!!,자급제 처음인데 진작에 이렇게 살껄 그랬어요 너무 좋아요!!,https://search.shopping.naver.com/catalog/2085...
